In [ ]:
#Mohammed Ennejmy
#Data assignment for Data Analytics Engineer at ResQ Club 

#Answers:

#Deliverable #1

#This SQL query will create a view that joins the orders, users, and providers tables into one unified presentation table.:


CREATE VIEW presentation_table AS
SELECT
    o.id AS order_id,
    o.createdAt AS order_date,
    o.userId as customerId,
    o.providerId as partnerId,
    o.sales,
    o.quantity,
    o.currency,
    u.country AS user_country,
    u.registeredDate AS user_registered_date,
    p.defaultOfferType AS partner_segment,
    p.country AS provider_country,
    p.registeredDate AS partner_registered_date
FROM
    orders o
    JOIN users u ON o.userId = u.id
    JOIN providers p ON o.providerId = p.id;
—————————————————————————

#For the Analyst:)

-- Top 10 partners by sales
SELECT partnerId, SUM(sales) AS total_sales
FROM presentation_table
GROUP BY partnerId
ORDER BY total_sales DESC
LIMIT 10;

—————————————————————————

-- Customers' favourite partner segments
SELECT partner_segment, COUNT(order_id) AS total_orders
FROM presentation_table
GROUP BY partner_segment
ORDER BY total_orders DESC;


#################################################################################3
#################################################################################3

#Deliverable #2
# For this I used Panda to create the Presentation table instead of Sql query

# loading in modules
import pandas as pd
import sqlite3

# creating file path
dbfile = '/Users/mohammed.ennejmy/Downloads/mock_resq.db'
# Create a SQL connection to our SQLite database
conn = sqlite3.connect(dbfile)

# creating cursor
cur = conn.cursor()
# reading all table names
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
print(table_list)

# fetch table from db and rename 'id'
orders = pd.read_sql_query('select * from orders', conn)
orders.head(10)
orders.rename({'id': 'orderId', 'userId':'customerId', 'providerId':'partnerId'}, axis=1, inplace=True)
orders.head()

# fetch table from db 
customers = pd.read_sql_query('select * from users', conn)
customers.head()
#customers.rename({'id':'customerId'}, axis = 1, inplace = True)
customers.head()

# fetch table from db 
partners = pd.read_sql_query('select * from providers', conn)
partners.head()
#partners.rename({'id':'partnerId'}, axis = 1, inplace = True)
partners.head()

# joining the oreder, customers and partners table into presentation table
presentation_table = pd.merge(orders, customers, left_on = 'customerId', right_on = 'id', how = 'inner').drop(columns=['id'])\
.merge(partners, left_on = 'partnerId', right_on = 'id', how ='inner').drop(columns=['id'])
# presentation_table = presentation_table[presentation_table['refunded'] == 1]
# presentation_table['refunded'].dtypes
# presentation_table['refunded'] = presentation_table['refunded'].astype('int32')
# presentation_table.sort_values('refunded', ascending = False)

# - Average Order Value
# presentation_table['sales'].dtypes
presentation_table['sales'] = pd.to_numeric(presentation_table['sales'], errors = 'coerce')
presentation_table.head()
avg_order_value = presentation_table['sales'].mean()
avg_order_value

# - Purchase Frequency 
purchase_frequency = presentation_table.groupby('customerId')['orderId']
purchase_frequency

# Customer Lifespan 
# Convert order_date to datetime format
presentation_table['createdAt'].dtypes
presentation_table['createdAt'] = pd.to_datetime(presentation_table['createdAt'])
# Calculate the first and last order date per customer
presentation_table['first_order'] = presentation_table.groupby('customerId')['createdAt'].transform('min') 
presentation_table['last_order'] = presentation_table.groupby('customerId')['createdAt'].transform('max')
presentation_table.head()
# Calculate the lifespan in months between first and last orders
presentation_table['customer_lifespan'] = (presentation_table['last_order'] - presentation_table['first_order']).dt.days/30

avg_customer_lifespan = presentation_table['customer_lifespan'].mean()
print(f"Average Customer Lifespan: {avg_customer_lifespan} months")

# Calculate CLV 
clv = avg_order_value * purchase_frequency * avg_customer_lifespan
print(f"Customer Lifetime Value (CLV): {clv}")
conn.close()